In [9]:
import requests
import pandas as pd  
import os

## Camada Bronze do datalake

Extraindo os dados proveniente do data sus. Vou salvar esses dados na camada bronze do datalake

In [10]:
YEAR = 2025
url = (
        "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/"
        "SRAG/2025/INFLUD25-10-11-2025.csv"
)

response = requests.get(url)
response.raise_for_status()

df = pd.read_csv(url, sep=';', encoding='latin1')
df.head()

/tmp/ipykernel_2939/257838647.py:10: DtypeWarning: Columns (14,92,94,185,186,188,189) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep=';', encoding='latin1')


,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,...,VG_OMS,VG_OMSOUT,VG_LIN,VG_MET,VG_METOUT,VG_DTRES,VG_ENC,VG_REINF,VG_CODEST,REINF
0,31735451969802,2024-12-29,1,2024-12-29,1,SP,GVE VIII MOGI DAS CRUZES,1333.0,GUARULHOS,351880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,31735516593521,2024-12-29,1,2024-12-29,1,SP,GVE VIII MOGI DAS CRUZES,1333.0,GUARULHOS,351880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,31735544021940,2024-12-29,1,2024-12-29,1,SP,GVE XXIX SAO JOSE DO RIO PRETO,1354.0,SAO JOSE DO RIO PRETO,354980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,31735559043222,2024-12-30,1,2024-12-29,1,AC,REGIONAL DO BAIXO ACRE,1938.0,RIO BRANCO,120040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,31735560267779,2024-12-30,1,2024-12-29,1,PB,I NRS JOAO PESSOA,1377.0,JOAO PESSOA,250750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


## Camada Silver do datalake

O dataset do DATASUS contém diversas colunas relacionadas a diferentes doenças e condições de saúde. Como o problema abordado neste projeto refere-se à análise da Síndrome Respiratória Aguda Grave (SRAG), optou-se por selecionar apenas as colunas relevantes para esse contexto.  

As variáveis escolhidas incluem: a data de notificação da doença (DT_NOTIFIC); a informação sobre contato do paciente com aves ou suínos, indicando possível transmissão zoonótica (AVE_SUINO); a presença de febre (FEBRE); a ocorrência de falta de ar (DISPNEIA); a existência de desconforto respiratório (DESC_RESP); a presença de fatores de risco (FATOR_RISC); o status de vacinação (VACINA); o uso de antivirais (ANTIVIRAL); a internação em unidade de terapia intensiva (UTI); a evolução do quadro clínico do paciente (EVOLUCAO); e a ocorrência de surto de síndrome gripal (SURTO_SG).

Como o dataset é espelhado em uma ficha de enfermaria, os campos que não são marcados são iguais à Not a Number(nan) por isso que optei por subsituir esse campo por 0, também retirei os dados duplicados do dataset. O intuito desse tipo de tratamento é garantir a integridade dos dados e evitar que o modelo tire falsas conclusões a partir de uma análise de fontes errôneas de dados. Como essa primeira fase do tratamento é para retirar colunas relevantes e fazer um tratamento de dados mais simples, vou salvar esses dados na camada Silver do Datalake

In [17]:
columns_important = [
        'DT_NOTIFIC', 'AVE_SUINO', 'FEBRE',
        'DISPNEIA', 'DESC_RESP','FATOR_RISC',
        'VACINA', 'ANTIVIRAL', 'UTI','EVOLUCAO','SURTO_SG'
]

df_raw_important = df[columns_important]
df_treated_nan = df_raw_important.fillna(0)
df_treated = df_treated_nan.drop_duplicates() 

In [ ]:
df_treated_nan.head()

,DT_NOTIFIC,AVE_SUINO,FEBRE,DISPNEIA,DESC_RESP,FATOR_RISC,VACINA,ANTIVIRAL,UTI,EVOLUCAO,SURTO_SG
0,2024-12-29,2.0,0.0,1.0,0.0,1.0,2.0,2.0,2.0,1.0,2.0
1,2024-12-29,2.0,2.0,1.0,1.0,0.0,2.0,2.0,0.0,1.0,2.0
2,2024-12-29,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0
3,2024-12-30,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,0.0
4,2024-12-30,9.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,9.0


: 

## Camada Gold do datalake

Como uma grande parte das colunas desse dataset apresentam números como fonte de dados, optei por subsituir essa representação númerica por uma fonte textual. O meu intuito com esse tramanento é dimuir a alucinação da LLM ou que ela tenha maiores dificuldades em fornecer uma boa resposta por não compreender o significado da representação númerica nos valores das colunas. Nessa parte do datalake, os dados já estão refinados e prontos para serem utilizados por modelos de LLM

In [13]:
map_yes_no = {
        0: "Campo vazio", 1: "Sim", 2: "Não", 9: "Ignorado"
}
map_birds_pigs = {
        0: "Campo vazio", 1: "Sim, aves e/ou suínos", 2: "Não, nenhum",
        3: "Sim, outros", 9: "Ignorado"
    }
map_evolucao = {
        0: "Campo vazio", 1: "Cura", 2: "Óbito", 3: "Óbito por outras causa", 9: "Ignorado"
    }

columns_yes_no = [
        'FEBRE', 'DISPNEIA', 'DESC_RESP', 
        'FATOR_RISC', 'VACINA', 'ANTIVIRAL', 'UTI', 'SURTO_SG'
    ]


df_treated[columns_yes_no] = df_treated[columns_yes_no].apply(lambda col: col.map(map_yes_no))

df_treated['AVE_SUINO'] = df_treated['AVE_SUINO'].map(map_birds_pigs)
df_treated['EVOLUCAO'] = df_treated['EVOLUCAO'].map(map_evolucao)

df_treated['DT_NOTIFIC'] = pd.to_datetime(df_treated['DT_NOTIFIC'], errors='coerce') \
        .dt.strftime('%Y-%m-%d')   

df_treated.head()

/tmp/ipykernel_2939/2631345720.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_treated[columns_yes_no] = df_treated[columns_yes_no].apply(lambda col: col.map(map_yes_no))
/tmp/ipykernel_2939/2631345720.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_treated['AVE_SUINO'] = df_treated['AVE_SUINO'].map(map_birds_pigs)
/tmp/ipykernel_2939/2631345720.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,DT_NOTIFIC,AVE_SUINO,FEBRE,DISPNEIA,DESC_RESP,FATOR_RISC,VACINA,ANTIVIRAL,UTI,EVOLUCAO,SURTO_SG
0,2024-12-29,"Não, nenhum",Campo vazio,Sim,Campo vazio,Sim,Não,Não,Não,Cura,Não
1,2024-12-29,"Não, nenhum",Não,Sim,Sim,Campo vazio,Não,Não,Campo vazio,Cura,Não
2,2024-12-29,"Não, nenhum",Não,Sim,Sim,Sim,Não,Sim,Sim,Óbito,Não
3,2024-12-30,"Não, nenhum",Não,Não,Sim,Sim,Não,Não,Sim,Cura,Campo vazio
4,2024-12-30,Ignorado,Sim,Não,Não,Sim,Sim,Não,Não,Cura,Ignorado


## Gerando a saída dos dados

In [14]:
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

datalake_dir = os.path.join(project_root, "datalake", "gold")

os.makedirs(datalake_dir, exist_ok=True)

output_path = os.path.join(datalake_dir, f"srag_gold_notebook_{YEAR}.csv") 

df_treated.to_csv(output_path, index=False)